In [1]:
import kotoTelematics as kt

In [ ]:
"""

1) Instantiate a class of telematics, which generates the headers that will be used to call the API

2) Either set argument access_token to your generated access token OR Create a text file in your working directory, and put your access token in there, and then set access_token_file to the text file name containing your token

"""

In [2]:
#Store your telematics object
    #See __init__ method doc string for more info
kt_obj = kt.telematics(access_token_file="access_token.txt")

In [3]:
"""
Now your telematics object is created, it has class methods: vin_table_request, ts_table_request, ts_table_scrape

The vin_table_request method, makes a single call to the API, and only accepts fetch_size and columns as parameters.

The ts_table_request method, make a single call to the API, and accepts fetch_size, columns, as well as min and max timestamps as parameters. Because timestamp is returned in UTC, in milliseconds, the function also formats the timestamp for EST. Our data spans beyond that, but for our purposes we will only be considering data within the Columbus region.

The ts_table_scrape method, uses the ts_table_request method, it makes a call to the API, passing your provided start time and date. Additionally, this method needs a window of time specified in minutes. This creates a max time and max date parameter which get used by the ts_table_request method.
"""

'\nNow your telematics object is created, it has class methods: vin_table_request, ts_table_request, ts_table_scrape\n\nThe vin_table_request method, makes a single call to the API, and only accepts fetch_size and columns as parameters.\n\nThe ts_table_request method, make a single call to the API, and accepts fetch_size, columns, as well as min and max timestamps as parameters. Because timestamp is returned in UTC, in milliseconds, the function also formats the timestamp for EST. Our data spans beyond that, but for our purposes we will only be considering data within the Columbus region.\n\nThe ts_table_scrape method, uses the ts_table_request method, it makes a call to the API, passing your provided start time and date. Additionally, this method needs a window of time specified in minutes. This creates a max time and max date parameter which get used by the ts_table_request method.\n'

In [ ]:
"""
Using the Destination Collector as an example, the destination collector needs the following columns:
'vin', 'navigation_location_coordinate_longitude', 'navigation_location_coordinate_latitude', 'sequence'
"""

In [4]:
#Specify the columns that your query function will use
columns = ['vin', 'navigation_location_coordinate_longitude', 'navigation_location_coordinate_latitude', 'sequence']

In [11]:
#Example using the vin_table_request
vin_response_object = kt_obj.vin_table_request(fetch_size=5, columns=columns)

In [13]:
#As you can see not every value is going to have the the value we're looking for, this is a feature of the onboard telematics data itself, where it only populates some values, in a future version of the API this will be collated to remove unneccesary records, right now the data in it's rawest form

vin_response_object

[{'vin': '0001556fa4a96efdfb0e6f9c079d2fa5',
  'navigation_location_coordinate_longitude': None,
  'navigation_location_coordinate_latitude': None,
  'sequence': 38,
  'timestamp': 1578884537680},
 {'vin': '0001556fa4a96efdfb0e6f9c079d2fa5',
  'navigation_location_coordinate_longitude': None,
  'navigation_location_coordinate_latitude': None,
  'sequence': 38,
  'timestamp': 1578884609995},
 {'vin': '0001556fa4a96efdfb0e6f9c079d2fa5',
  'navigation_location_coordinate_longitude': None,
  'navigation_location_coordinate_latitude': None,
  'sequence': 38,
  'timestamp': 1578884610501},
 {'vin': '0001556fa4a96efdfb0e6f9c079d2fa5',
  'navigation_location_coordinate_longitude': -96.7078114,
  'navigation_location_coordinate_latitude': 33.1249199,
  'sequence': 38,
  'timestamp': 1578884639887},
 {'vin': '0001556fa4a96efdfb0e6f9c079d2fa5',
  'navigation_location_coordinate_longitude': None,
  'navigation_location_coordinate_latitude': None,
  'sequence': 38,
  'timestamp': 1578884651770}]

In [16]:
#Example using the vin_table_request
    #Note that timestamp doesn't need to be specified as a selected column, the function will add it itself if not specified
ts_response_object = kt_obj.ts_table_request(fetch_size=3, columns=columns, min_date='2020-01-12', max_date='2020-01-13')

In [17]:
ts_response_object

[{'vin': '0001556fa4a96efdfb0e6f9c079d2fa5',
  'navigation_location_coordinate_longitude': None,
  'navigation_location_coordinate_latitude': None,
  'sequence': 38,
  'timestamp': '2020-01-12 22:02:17 EST'},
 {'vin': '0001556fa4a96efdfb0e6f9c079d2fa5',
  'navigation_location_coordinate_longitude': None,
  'navigation_location_coordinate_latitude': None,
  'sequence': 38,
  'timestamp': '2020-01-12 22:03:29 EST'},
 {'vin': '0001556fa4a96efdfb0e6f9c079d2fa5',
  'navigation_location_coordinate_longitude': None,
  'navigation_location_coordinate_latitude': None,
  'sequence': 38,
  'timestamp': '2020-01-12 22:03:30 EST'}]

In [7]:
#Example Using ts_table_scrape with demo function
    #See doc string for more detailed info
ts_scrape_response_object = kt_obj.ts_table_scrape(\
    fetch_size=5, \
    #This specifies how many times the window should be pushed forward
    n_loops=5, \
    columns=columns, \
    start_date="2020-01-12", \
    #This specifies the range of the time window to be called in minutes
    time_window=10, \
    #This applies the destination collector static method to every response
    functions=[kt_obj.destination_collector_demo])

#Any functions like "destination_collector_demo" need to be written to iteratively aggregate the answer
#Meaning, that it needs to be able see a parse a single response from the API for the useful information
#Then it needs to store and update the intermediate data structure as per a data handler

{'099bb2badfc668ae2b230e4ddc44c2c1': {'540': [40.0634273, -82.9526528]}}
{'099bb2badfc668ae2b230e4ddc44c2c1': {'540': [40.0635917, -82.9526392]}}
{'099bb2badfc668ae2b230e4ddc44c2c1': {'540': [40.0637569, -82.9526265]}}
{'065b3e933a8e935a71284dfed5d41a38': {'136': [40.0291941, -83.1205772]}}
{'065b3e933a8e935a71284dfed5d41a38': {'136': [40.0293109, -83.1205999]}}
{'065b3e933a8e935a71284dfed5d41a38': {'136': [39.9732983, -83.1528167]}}


In [ ]:
"""
As you can see from the output, it collects the sequence and location by vin
Then the data handler function stores in the data, where it contionously updates the location for the sequence+vin pair
Because our data is arranged by timestamp, this means that the last location for a sequence+vin will be it's destination
"""